В этой статье я расскажу, как провел сравнение Keras и PyTorch на CV-задаче: распознавание CIFAR10 сетью MobileNetV2. Сравнение проводилось по двум критериям:
1. Объем кода
2. Производительность

## Загрузка моделей и подготовка данных

Загрузим модели MobileNetV2 из Keras и PyTorch и убедимся, что в них равное количество параметров. Для Keras будем делить кол-во параметров в слоях BatchNormalization на 2, поскольку running_mean и running_variance в Keras считаются параметрами, в отличие от PyTorch.

In [ ]:
from tensorflow import keras
keras_mobileNetV2 = keras.applications.MobileNetV2(include_top=False, input_shape=(32, 32, 3))
def get_params_count(layer):
  count = sum([len(x.flat) for x in layer.get_weights()])
  if isinstance(layer, keras.layers.BatchNormalization):
    return count // 2
  return count
print('Parameters:', sum([get_params_count(l) for l in keras_mobileNetV2.layers]))

import torch
torch.hub._validate_not_a_forked_repo = lambda *args: None #bugfix: https://github.com/pytorch/vision/issues/4156
torch_mobileNetV2 = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
print('Parameters:', sum([len(x.view(-1)) for x in torch_mobileNetV2.features.parameters()]))

Подготовим данные. Для моделей из Tensorflow и из PyTorch требуется разная нормализация данных. Также в PyTorch требуется поменять порядок осей.

In [ ]:
import tensorflow as tf, numpy as np, time
from tensorflow.keras.applications import imagenet_utils

(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train[:, 0]
y_test = y_test[:, 0]

X_train_keras = imagenet_utils.preprocess_input(X_train, mode='tf')
X_test_keras = imagenet_utils.preprocess_input(X_test, mode='tf')
X_train_torch = imagenet_utils.preprocess_input(X_train, mode='torch')
X_test_torch = imagenet_utils.preprocess_input(X_test, mode='torch')
X_train_torch = np.moveaxis(X_train_torch, 3, 1)
X_test_torch = np.moveaxis(X_test_torch, 3, 1)

## Пишем код для решения задачи на Keras

In [17]:
from tensorflow.keras import Sequential, layers, losses, optimizers

!nvidia-smi -L # check video card
model = Sequential([
  keras_mobileNetV2,
  layers.GlobalMaxPool2D(),
  layers.Dropout(0.5),
  layers.Dense(10)
])
model.compile(loss=losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=optimizers.Adam(1e-4), metrics='accuracy')
start_time = time.time()
model.fit(X_train_keras, y_train, validation_data=(X_test_keras, y_test),
          batch_size=128, epochs=10)
print(time.time() - start_time)

GPU 0: A100-SXM4-40GB (UUID: GPU-67903482-db10-8d14-1b00-7d7dd6d651bb)
Epoch 1/10
391/391 [==============================] - 15s 32ms/step - loss: 1.3689 - accuracy: 0.5449 - val_loss: 1.9892 - val_accuracy: 0.2680
Epoch 2/10
391/391 [==============================] - 11s 29ms/step - loss: 1.0378 - accuracy: 0.6538 - val_loss: 1.6533 - val_accuracy: 0.3787
Epoch 3/10
391/391 [==============================] - 11s 29ms/step - loss: 0.9003 - accuracy: 0.6981 - val_loss: 1.3371 - val_accuracy: 0.5239
Epoch 4/10
391/391 [==============================] - 12s 30ms/step - loss: 0.7913 - accuracy: 0.7290 - val_loss: 1.1792 - val_accuracy: 0.6190
Epoch 5/10
391/391 [==============================] - 11s 29ms/step - loss: 0.7130 - accuracy: 0.7539 - val_loss: 1.1181 - val_accuracy: 0.6634
Epoch 6/10
391/391 [==============================] - 12s 29ms/step - loss: 0.6273 - accuracy: 0.7809 - val_loss: 1.0269 - val_accuracy: 0.7002
Epoch 7/10
391/391 [==============================] - 11s 29ms/st

## Пишем код для решения задачи на PyTorch

Используем библиотеку torchmetrics для удобного расчета и накопления метрики accuracy.

In [5]:
!pip install torchmetrics -q

from tqdm.notebook import tqdm
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics

cuda = torch.device('cuda')

batch_size = 128
train_dataset = TensorDataset(torch.Tensor(X_train_torch).to(cuda), torch.LongTensor(y_train).to(cuda))
test_dataset = TensorDataset(torch.Tensor(X_test_torch).to(cuda), torch.LongTensor(y_test).to(cuda))
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = torch_mobileNetV2
model.classifier[1] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
)
model.to(cuda)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

start_time = time.time()
for epoch in range(10):
    model.train()
    epoch_sum_loss = 0
    train_accuracy = torchmetrics.Accuracy().to(cuda)
    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=len(train_loader)):
      optimizer.zero_grad()
      output = model(data)
      loss = loss_fn(output, target)
      loss.backward()
      optimizer.step()
      train_accuracy.update(output, target)
      epoch_sum_loss += loss.item() * data.size(0)
    model.eval()
    val_sum_loss = 0
    val_accuracy = torchmetrics.Accuracy().to(cuda)
    for batch_idx, (data, target) in enumerate(test_loader):
      output = model(data)
      val_accuracy.update(output, target)
      val_sum_loss += loss.item() * data.size(0)

    average_loss = epoch_sum_loss/len(train_loader.dataset)
    average_val_loss = val_sum_loss/len(test_loader.dataset)
    print(f'loss {average_loss:g} train_accuracy {train_accuracy.compute().item():g}'
          f' val_loss {average_val_loss:g}, val_accuracy {val_accuracy.compute().item():g}')
print(time.time() - start_time)


loss 1.30399 train_accuracy 0.53986 val_loss 0.862852, val_accuracy 0.6851



loss 0.82052 train_accuracy 0.71386 val_loss 0.620808, val_accuracy 0.7419



loss 0.630399 train_accuracy 0.779 val_loss 0.431304, val_accuracy 0.7533



loss 0.482806 train_accuracy 0.83042 val_loss 0.248696, val_accuracy 0.7571



loss 0.35488 train_accuracy 0.87766 val_loss 0.107332, val_accuracy 0.7574



loss 0.250915 train_accuracy 0.91436 val_loss 0.0915811, val_accuracy 0.7605



loss 0.173062 train_accuracy 0.94076 val_loss 0.0508658, val_accuracy 0.7596



loss 0.142785 train_accuracy 0.9508 val_loss 0.0469299, val_accuracy 0.7601



loss 0.112422 train_accuracy 0.96188 val_loss 0.0688462, val_accuracy 0.7683



loss 0.092886 train_accuracy 0.96714 val_loss 0.0304079, val_accuracy 0.7619
152.94082808494568


## Сравнение по объему кода

Как видим, в PyTorch нужно писать намного больший объем кода. Есть правда высокоуровневая библиотека pytorch-lightning (и библиотека torchbearer, , которая позволяет писать код в стиле Keras.

Можно возразить, что за счет объема кода в PyTorch доступен больший контроль за процессом обучения, однако это не так. В Keras также можно писать [кастомный цикл обучения](https://keras.io/guides/customizing_what_happens_in_fit/) и [кастомные модели](https://keras.io/guides/making_new_layers_and_models_via_subclassing/) в виде подклассов. Но если вам достаточно стандартных моделей и циклов обучения, то Keras избавляет от необходимости все подробно расписывать. За счет этого код получается более лаконичным, что по моему мнению крайне важно.

## Сравнение по скорости обучения

Используем размер батча 128 и видеокарту Nvidia A100. Для каждого теста создаем виртуальную машину на vasi.ai, используя соответствующий docker-образ (tensorflow или pytorch).

По производительности тоже выигрывает Tensorflow+Keras: 10 эпох занимают 118 секунд. В PyTorch 10 эпох заняли 153 секунды.

Иногда в Keras в конце последней эпохи происходит длительная задержка, до 40 секунд. Ранее я такого не встречал, видимо это баг в новой версии Tensorflow 2.6. В Tensorflow в новых версиях нередко появляются баги, это конечно плохо, но с другой стороны говорит о том, что библиотека развивается и в ней что-то меняется.

Для сравнения, в Keras с параметром компиляции модели [run_eagerly=True](https://keras.io/getting_started/intro_to_keras_for_engineers/#debugging-your-model-with-eager-execution) (то есть динамический граф вычислений) каждая эпоха занимает около 40 секунд. Интересно, что в Google Colab с видеокартой Tesla P100 (которая по идее менее мощная) обучение на Tensorflow занимает меньше времени, чем с видеокартой A100 в vast.ai. Это может быть связано с хорошей оптимизацией TF в Colab.

## Обработка исключений и сложность написания кода

Важным критерием сравнения библиотек является легкость отладки кода. Проведенный выше тест не позволяет сравнить по этому критерию. Однако из предыдущего опыта скажу, что Tensorflow часто выдает непонятные сообщения об ошибках. Например, такой код выдает следующую ошибку:

In [13]:
import tensorflow as tf
class Network(Sequential):
  def __init__(self, *args):
    super(Network, self).__init__()
    self.train_loss_per_batch = []
    self.train_accuracy_per_batch = []
  def train_step(self, batch):
    X, y = batch
    with tf.GradientTape() as tape:
      y_pred = self(X, training=True)
      loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

model = Network([layers.Dense(10)])
model.compile(loss=losses.SparseCategoricalCrossentropy(True), optimizer='adam', metrics='accuracy')
X, y = np.zeros((100, 100)), np.zeros(100)
model.fit(X, y, epochs=1, verbose=0)

TypeError: ignored

Пытаться понять смысл ошибки по тексту и колл-стеку безнадежно, приходится просматривать код, исправлять что-то наугад, упрощать код до тех пор, пока ошибка не исчезнет, и таким путем ее диагностировать. Например, причина приведенной выше ошибки была в отсутствии возвращаемого значения из функции train_step.

Такая особенность Tensorflow и Keras является платой за вычислительную оптимизацию. В Tensorflow и Keras встроены сложные механизмы обработки python-кода и трансляции его в байткод, из-за чего возникающие ошибки иногда плохо интерпретируемы. Исходный код Tensorflow очень сложен, из-за чего в новых версиях Tensorflow часто возникают баги.

Приведу еще один недавний пример из практики, демонстрирующий сложность написания кода на Keras для статического графа вычислений. Мне нужно было написать кастомную стратегию управления learning rate. callbacks.LearningRateScheduler мне не подошел, поскольку в нем learning rate фиксирован в течение каждой эпохи. Поэтому мне понадобилось создавать подкласс класса schedules.LearningRateSchedule:

```
class LRSchedule(LearningRateSchedule):
    def __call__(self, step):
        return schedule_func(step)
schedule_object = LRSchedule()
```

После запуска модели Tensorflow начал ругаться на то, что "тензор не может использоваться как bool". Оказалось, что проблема в том, что schedule_func является обычной питоновской функцией. Чтобы встроить такую функцию в граф вычислений, нужно выбрать один из трех вариантов:

1. Сделать весь граф динамическим, что не вариант, так как сильно ухудшает производительность
2. Преобразовать функцию в tf.function, то есть, упрощенно говоря, скомпилировать ее в C++
3. Преобразовать функцию в tf.py_function, чтобы из графа вычислений делался вызов интерпретатора python

В третьем варианте по идее нужно делать так:

`schedule_func = tf.py_function(func=schedule_func, inp=[inputs], Tout=tf.float64)`

Однако здесь нужно явно указывать входные тензоры (inputs). Непонятно что будет являться входным тензором внутри `LRSchedule.__call__`. Поэтому этот вариант я не смог реализовать. 

Другим вариантом является добавление аннотации `@tf.function` в том месте, где определяется функция schedule_func. Таким образом schedule_func будет скомпилирована в С++ с помощью `tf.autograph`. Я попробовал так сделать. Внутри функции schedule_func есть блок if-else, и Tensorflow начинает ругаться на то, что в ветке if возвращается значение float32, а в ветке else значение float64. Мне понадобилось полчаса чтобы победить эту проблему: оказалось, что достаточно обернуть константы, используемые в schedule_func, в вызов `np.float64()`. Когда мне показалось, что я решил эту проблему, я запустил обучение, и через некоторое время та же ошибка возникла снова. Оказалось, что предварительный тест не всегда ее выявляет, то есть Tensorflow компилирует функцию, но в методе .fit() она не работает. И чтобы решить эту проблему, нужно очень внимательно следить за типами используемых переменных. Если при тестировании функция работала с аргументом типа float64, то это еще не означает, что при обучении она будет работать с аргументом типа float32. Впрочем возможно, что я делаю что-то не так, но учитывая мой длительный опыт работы с Tensorflow все это говорит о чрезмерной усложненности этой библиотеки.

## Обсуждение

Распространенным заблуждением является то, что Keras - верхнеуровневая библиотека, которая ограничивает ваши возможности, пряча многое "под капотом". На самом деле Keras следует концепции постепенного усложнения, то есть позволяет выполнять кастомизацию разного уровня, вплоть до самой полной. При желании в Keras можно писать код точно так же, как в PyTorch: программируя модели как подклассы, определяя для них метод call (в PyTorch он называется forward), программируя кастомный цикл оптимизации. При этом в Keras есть то, чего нет в PyTorch - возможности выбора типа графа вычислений (статический или динамический).

PyTorch стал популярен из-за того, что в нем удобнее решать сложные задачи, используя интуитивно понятный динамический граф вычислений. Отток разработчиков из Tensorflow+Keras в PyTorch (см. [график популярности](https://trends.google.ru/trends/explore?cat=1227&date=today%205-y&q=Keras,TensorFlow,PyTorch)) начался еще в эпоху версии Tensorflow 1.x, которая действительно была очень запутанной. В сознании многих пользователей Tensorflow все еще ассоциируется с версией 1.x. Очень много кода на Github написано именно для этой версии и не работает в версии 2.x, что негативно сказывается на репутации библиотеки.

Сейчас ситуация отчасти поменялась, поскольку вышел Tensorflow 2.x с возможностью построения динамического графа. Но тем не мене динамический граф на Tensorflow, судя по всему, работает существенно медленнее, чем на PyTorch. Кроме того, если вы делаете на Tensorflow что-то нестандартное, то у вас могут возникнуть ошибки, и вам трудно будет понять их причину.

### Выводы

Здесь я попробую описать схему принятия решения о том, какую библиотеку использовать. Возможно с получением нового опыта я буду дополнять или корректировать этот раздел. Итак, выбор библиотеки зависит от следующих критериев:

**Сложность задачи.** Если вы делаете достаточно простые и стандартные вещи, то TensorFlow+Keras будет хорошим выбором. Код на Keras обычно получается более лаконичным: вы пишете только то, что не является стандартным - остальное за вас делается автоматически. Но и в PyTorch есть удобная верхнеуровневая библиотека torchbearer, которая позволяет писать код в стиле Keras.

Если же ваша задача сложная и нестандартная, то вам нужно решить, насколько для вас важна производительность, масштабируемость и переносимость кода. Если эти вещи для вас критичны, то лучше строить статический граф на Tensorflow, хотя при этом могут возникнуть трудно понимаемые ошибки. В противном случае лучше использовать PyTorch, особенно если у вас недостаточно опыта работы с Tensorflow. PyTorch будет особенно подходящим если вы хотите детально расписывать весь код оптимизации в императивном стиле, поскольку динамический граф вычислений на нем выполняется быстрее.

**Развертывание нейросетей в production.** TensorFlow предоставляет широкие возможности для production'а, такие как TensorFlow.js и TensorFlow Lite, но об аналогах этих библиотек для PyTorch я увы не располагаю информацией. В целом статичный граф вычислений абстрагирует архитектуру от языка python и делает ее более переносимой.

**Необходимость использования предобученных моделей.** Набор предобученных сетей, доступный для PyTorch и TensorFlow, различается. Например, если нужная вам предобученная сеть доступна только для PyTorch, то вам придется использовать PyTorch, если вы не хотите лишних сложностей с конвертированием моделей.

**Опыт в использовании той или иной библиотеки.** На более знакомой вам библиотеке вам будет проще писать код. Но с другой стороны используя менее знакомую библиотеку вы приобретаете новый ценный опыт.